In [1]:
# import all relevant libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

import requests
import json
from sec_api import QueryApi

import yfinance as yf
from sec_cik_mapper import StockMapper

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_rows', None)

I am using base code from https://gist.github.com/janlukasschroeder/3da274150fd00c1c1776c7e541a9b61b to get me started in scraping financial statements from the SEC. I will be using 10-K forms (annual SEC filing) rather than all quarterly filings (10-Q) to reduce the size of data.

In [27]:
from sec_api import QueryApi

api_key = '88140c3a554da7220ba7453a5e1ea2221c5f1e0e42c3293bbe75857245774e2e'

# get your API key at https://sec-api.io
query_api = QueryApi(api_key=api_key)

# fetch all 10-K filings for Apple from the last 25 years to test this API
query = {
    "query": {
        "query_string": {
            "query": "filedAt:[2011-01-01 TO 2012-01-01] AND (formType:\"10-K\")"
        }
    },
    "from": "0",
    "size": "100",
    "sort": [{ "filedAt": { "order": "desc" } }]
}

query_result_100 = query_api.get_filings(query)

In [28]:
# XBRL-to-JSON converter API endpoint
xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"

In [78]:
query_result_100

{'total': {'value': 10000, 'relation': 'gte'},
 'query': {'from': 0, 'size': 100},
 'filings': [{'id': '53be96c6c4517dfc6465bc2f0ab7b2aa',
   'accessionNo': '0001144204-11-072464',
   'cik': '1435530',
   'ticker': '',
   'companyName': 'China Polypeptide Group, Inc.',
   'companyNameLong': 'China Polypeptide Group, Inc. (Filer)',
   'formType': 'NT 10-K',
   'description': 'Form NTN 10K - Notices of Late Filings of Form 10-K or 10-KSB',
   'filedAt': '2011-12-30T21:03:48-05:00',
   'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1435530/000114420411072464/0001144204-11-072464.txt',
   'linkToHtml': 'https://www.sec.gov/Archives/edgar/data/1435530/0001144204-11-072464-index.htm',
   'linkToXbrl': '',
   'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/1435530/000114420411072464/v244435_nt10k.htm',
   'entities': [{'companyName': 'China Polypeptide Group, Inc. (Filer)',
     'cik': '0001435530',
     'irsNo': '208731646',
     'stateOfIncorporation': 'DE',
     'fis

In [79]:
accession_numbers = []

# extract accession numbers of each filing
for filing in query_result_100['filings']:
    accession_numbers.append(filing['accessionNo']);

accession_numbers

['0001144204-11-072464',
 '0001513162-11-000410',
 '0001393905-11-000854',
 '0001471242-11-000511',
 '0001354488-11-005251',
 '0001185185-11-002321',
 '0001013762-11-003420',
 '0001513162-11-000404',
 '0001144204-11-072364',
 '0001144204-11-072359',
 '0001078782-11-003814',
 '0001354488-11-005245',
 '0000930413-11-008132',
 '0001354488-11-005242',
 '0001102624-11-000811',
 '0001004878-11-000435',
 '0001437749-11-009914',
 '0001078782-11-003810',
 '0001437749-11-009912',
 '0001019687-11-003992',
 '0001437749-11-009909',
 '0001513162-11-000401',
 '0001079973-11-001036',
 '0001165527-11-001255',
 '0000072633-11-000030',
 '0001264931-11-000743',
 '0001091818-11-000653',
 '0001078782-11-003808',
 '0001078782-11-003805',
 '0001437749-11-009896',
 '0001213900-11-006882',
 '0001214659-11-004557',
 '0001165527-11-001252',
 '0001038838-11-000399',
 '0001127855-11-000735',
 '0001127855-11-000733',
 '0001144204-11-072245',
 '0001391609-11-000602',
 '0001078782-11-003800',
 '0001140361-11-059222',


In [80]:
import time

# get XBRL-JSON for a given accession number
def get_xbrl_json(accession_no, retry = 0):
    request_url = xbrl_converter_api_endpoint + "?accession-no=" + accession_no + "&token=" + api_key

    # linear backoff in case API fails with "too many requests" error
    try:
      response_tmp = requests.get(request_url)
      xbrl_json = json.loads(response_tmp.text)
    except:
      if retry > 5:
        raise Exception('API error')
      
      # wait 500 milliseconds on error and retry
      time.sleep(0.5) 
      return get_xbrl_json(accession_no, retry + 1)

    return xbrl_json

In [81]:
list_xbrl = []

for acc_no in accession_numbers:
    list_xbrl.append(get_xbrl_json(acc_no))

In [82]:
cleaned_list_xbrl = []

for xbrl in list_xbrl:
    if 'status' in xbrl.keys():
        pass
    else:
        cleaned_list_xbrl.append(xbrl)

In [83]:
len(cleaned_list_xbrl)

37

In [84]:
len(list_xbrl)

100

In [21]:
income_statement_final.T

,NaN,CostOfGoodsAndServicesSold,GrossProfit,ResearchAndDevelopmentExpense,SellingGeneralAndAdministrativeExpense,OperatingExpenses,OperatingIncomeLoss,NonoperatingIncomeExpense,IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,IncomeTaxExpenseBenefit,NetIncomeLoss,EarningsPerShareBasic,EarningsPerShareDiluted,WeightedAverageNumberOfSharesOutstandingBasic,WeightedAverageNumberOfDilutedSharesOutstanding,NaN,SalesRevenueNet,CommonStockDividendsPerShareDeclared
2014-09-28-2015-09-26,NaN,140089000000,93626000000,8067000000,14329000000,22396000000,71230000000,1285000000,72515000000,19121000000,53394000000,9.28,9.22,5753421000,5793069000,NaN,233715000000,1.98
2015-09-27-2016-09-24,NaN,131376000000,84263000000,10045000000,14194000000,24239000000,60024000000,1348000000,61372000000,15685000000,45687000000,8.35,8.31,5470820000,5500281000,NaN,215639000000,2.18
2016-09-25-2017-09-30,NaN,141048000000,88186000000,11581000000,15261000000,26842000000,61344000000,2745000000,64089000000,15738000000,48351000000,9.27,9.21,5217242000,5251692000,NaN,229234000000,2.4
2017-10-01-2018-09-29,NaN,163756000000,101839000000,14236000000,16705000000,30941000000,70898000000,2005000000,72903000000,13372000000,59531000000,12.01,11.91,4955377000,5000109000,265595000000,NaN,NaN
2018-09-30-2019-09-28,260174000000,161782000000,98392000000,16217000000,18245000000,34462000000,63930000000,1807000000,65737000000,10481000000,55256000000,11.97,11.89,4617834000,4648913000,NaN,NaN,NaN
2019-09-29-2020-09-26,274515000000,169559000000,104956000000,18752000000,19916000000,38668000000,66288000000,803000000,67091000000,9680000000,57411000000,3.31,3.28,17352119000,17528214000,NaN,NaN,NaN
2020-09-27-2021-09-25,365817000000,212981000000,152836000000,21914000000,21973000000,43887000000,108949000000,258000000,109207000000,14527000000,94680000000,5.67,5.61,16701272000,16864919000,NaN,NaN,NaN


In [49]:
cleaned_list_xbrl[6]

{'CoverPage': {'EntityRegistrantName': 'SINGLE TOUCH SYSTEMS INC',
  'DocumentType': '10-K',
  'DocumentPeriodEndDate': '2011-09-30',
  'AmendmentFlag': 'false',
  'EntityCentralIndexKey': '0001157817',
  'CurrentFiscalYearEndDate': '--09-30',
  'EntityCommonStockSharesOutstanding': {'unitRef': 'Shares',
   'decimals': 'INF',
   'period': {'instant': '2011-09-30'},
   'value': '130182392'},
  'EntityPublicFloat': {'unitRef': 'USD',
   'decimals': '0',
   'period': {'instant': '2011-09-30'},
   'value': '67573482'},
  'EntityFilerCategory': 'Smaller Reporting Company',
  'EntityCurrentReportingStatus': 'No',
  'EntityVoluntaryFilers': 'No',
  'EntityWellKnownSeasonedIssuer': 'No',
  'DocumentFiscalYearFocus': '2011',
  'DocumentFiscalPeriodFocus': 'FY'},
 'BalanceSheets': {'CashAndCashEquivalentsAtCarryingValue': [{'unitRef': 'USD',
    'decimals': '0',
    'period': {'instant': '2011-09-30'},
    'value': '523801'},
   {'unitRef': 'USD',
    'decimals': '0',
    'period': {'instant': '

In [5]:
# convert XBRL-JSON of balance sheet to pandas dataframe
def get_balance_sheet(xbrl_json):
    balance_sheet_store = {}

    for usGaapItem in xbrl_json['BalanceSheets']:
        values = []
        indicies = []

        for fact in xbrl_json['BalanceSheets'][usGaapItem]:
            # only consider items without segment.
            if 'segment' not in fact:
                index = fact['period']['instant']

                # avoid duplicate indicies with same values
                if index in indicies:
                    continue
                    
                # add 0 if value is nil
                if "value" not in fact:
                    values.append(0)
                else:
                    values.append(fact['value'])

                indicies.append(index)                    

            balance_sheet_store[usGaapItem] = pd.Series(values, index=indicies) 

    balance_sheet = pd.DataFrame(balance_sheet_store).reset_index()
    balance_sheet['index'] = balance_sheet['index'].str.slice(-10, -6)
    balance_sheet = balance_sheet.rename(columns={'index': 'year'})
    balance_sheet = balance_sheet.sort_values(by='year')
    balance_sheet['CIK'] = xbrl_json['CoverPage']['EntityCentralIndexKey']
    balance_sheet = pd.DataFrame(balance_sheet.iloc[-1, :]).T
    # switch columns and rows so that US GAAP items are rows and each column header represents a date instant
    return balance_sheet

In [86]:
test = get_balance_sheet(cleaned_list_xbrl[10])
test

,year,Cash,AccountsReceivableNetCurrent,AssetsCurrent,PropertyPlantAndEquipmentNet,Assets,AccountsPayableAndAccruedLiabilitiesCurrent,NotesPayableCurrent,Liabilities,CommonStockValue,AdditionalPaidInCapitalCommonStock,DevelopmentStageEnterpriseDeficitAccumulatedDuringDevelopmentStage,StockholdersEquity,LiabilitiesAndStockholdersEquity,CIK
4,2011,17998,6328,24326,1970,26296,132946,27780,160726,3895,65080,203405,-134430,26296,0001482179


In [87]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 4 to 4
Data columns (total 15 columns):
 #   Column                                                              Non-Null Count  Dtype 
---  ------                                                              --------------  ----- 
 0   year                                                                1 non-null      object
 1   Cash                                                                1 non-null      object
 2   AccountsReceivableNetCurrent                                        1 non-null      object
 3   AssetsCurrent                                                       1 non-null      object
 4   PropertyPlantAndEquipmentNet                                        1 non-null      object
 5   Assets                                                              1 non-null      object
 6   AccountsPayableAndAccruedLiabilitiesCurrent                         1 non-null      object
 7   NotesPayableCurrent           

In [88]:
test2 = get_balance_sheet(cleaned_list_xbrl[10])
test2

,year,Cash,AccountsReceivableNetCurrent,AssetsCurrent,PropertyPlantAndEquipmentNet,Assets,AccountsPayableAndAccruedLiabilitiesCurrent,NotesPayableCurrent,Liabilities,CommonStockValue,AdditionalPaidInCapitalCommonStock,DevelopmentStageEnterpriseDeficitAccumulatedDuringDevelopmentStage,StockholdersEquity,LiabilitiesAndStockholdersEquity,CIK
4,2011,17998,6328,24326,1970,26296,132946,27780,160726,3895,65080,203405,-134430,26296,0001482179


In [89]:
pd.concat([test, test2])

,year,Cash,AccountsReceivableNetCurrent,AssetsCurrent,PropertyPlantAndEquipmentNet,Assets,AccountsPayableAndAccruedLiabilitiesCurrent,NotesPayableCurrent,Liabilities,CommonStockValue,AdditionalPaidInCapitalCommonStock,DevelopmentStageEnterpriseDeficitAccumulatedDuringDevelopmentStage,StockholdersEquity,LiabilitiesAndStockholdersEquity,CIK
4,2011,17998,6328,24326,1970,26296,132946,27780,160726,3895,65080,203405,-134430,26296,0001482179
4,2011,17998,6328,24326,1970,26296,132946,27780,160726,3895,65080,203405,-134430,26296,0001482179


In [90]:
full_list = []

for xbrl in cleaned_list_xbrl:
    try:
        full_list.append(get_balance_sheet(xbrl))
    except:
        continue
len(full_list)    
balance_full_df = pd.concat(full_list)
balance_full_df.head()

/var/folders/lz/58xhrs5n36n2ng9kp4lk67t80000gp/T/ipykernel_2370/3092013937.py:26: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  balance_sheet_store[usGaapItem] = pd.Series(values, index=indicies)
/var/folders/lz/58xhrs5n36n2ng9kp4lk67t80000gp/T/ipykernel_2370/3092013937.py:26: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  balance_sheet_store[usGaapItem] = pd.Series(values, index=indicies)
/var/folders/lz/58xhrs5n36n2ng9kp4lk67t80000gp/T/ipykernel_2370/3092013937.py:26: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  balance_sheet_store[usGaapItem] = pd.Series(values, index=indicies)
/var/folders/lz/58xhrs5n36n2ng9kp4lk67t80000gp/T/ipykernel_23

,year,CashAndCashEquivalentsAtCarryingValue,NotesAndLoansReceivableNetCurrent,PrepaidExpenseAndOtherAssetsCurrent,AssetsCurrent,PropertyPlantAndEquipmentNet,Assets,AccountsPayableCurrent,EmployeeRelatedLiabilitiesCurrent,OtherLiabilitiesCurrent,...,UnrecoveredCostsForPurchasedGasAmount,DeferredCosts,UnamortizedDebtIssuanceExpense,OtherShortTermBorrowings,DepositLiabilitiesAccruedInterest,DividendsPayableCurrentAndNoncurrent,DeferredCompensationLiabilityClassifiedNoncurrent,PensionAndOtherPostretirementDefinedBenefitPlansLiabilitiesNoncurrent,DueToRelatedParties,OtherNotesPayable
2,2011,5097,47500,29299,81896,13273,95169,1794935,771473,573667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011,453243,NaN,NaN,714818,3964003,5545052,180031,NaN,75056,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011,2124406,NaN,NaN,20916648,5155528,30596000,3233597,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011,39188,NaN,8656,93842,12201,106043,333735,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,2011,NaN,NaN,NaN,75780,43068,204023,167420,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
balance_full_df.columns

Index(['year', 'CashAndCashEquivalentsAtCarryingValue',
       'NotesAndLoansReceivableNetCurrent',
       'PrepaidExpenseAndOtherAssetsCurrent', 'AssetsCurrent',
       'PropertyPlantAndEquipmentNet', 'Assets', 'AccountsPayableCurrent',
       'EmployeeRelatedLiabilitiesCurrent', 'OtherLiabilitiesCurrent',
       ...
       'UnrecoveredCostsForPurchasedGasAmount', 'DeferredCosts',
       'UnamortizedDebtIssuanceExpense', 'OtherShortTermBorrowings',
       'DepositLiabilitiesAccruedInterest',
       'DividendsPayableCurrentAndNoncurrent',
       'DeferredCompensationLiabilityClassifiedNoncurrent',
       'PensionAndOtherPostretirementDefinedBenefitPlansLiabilitiesNoncurrent',
       'DueToRelatedParties', 'OtherNotesPayable'],
      dtype='object', length=164)

In [92]:
balance_full_df

,year,CashAndCashEquivalentsAtCarryingValue,NotesAndLoansReceivableNetCurrent,PrepaidExpenseAndOtherAssetsCurrent,AssetsCurrent,PropertyPlantAndEquipmentNet,Assets,AccountsPayableCurrent,EmployeeRelatedLiabilitiesCurrent,OtherLiabilitiesCurrent,...,UnrecoveredCostsForPurchasedGasAmount,DeferredCosts,UnamortizedDebtIssuanceExpense,OtherShortTermBorrowings,DepositLiabilitiesAccruedInterest,DividendsPayableCurrentAndNoncurrent,DeferredCompensationLiabilityClassifiedNoncurrent,PensionAndOtherPostretirementDefinedBenefitPlansLiabilitiesNoncurrent,DueToRelatedParties,OtherNotesPayable
2,2011,5097,47500,29299,81896,13273,95169,1794935,771473,573667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011,453243,NaN,NaN,714818,3964003,5545052,180031,NaN,75056,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011,2124406,NaN,NaN,20916648,5155528,30596000,3233597,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011,39188,NaN,8656,93842,12201,106043,333735,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,2011,NaN,NaN,NaN,75780,43068,204023,167420,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011,104095000,NaN,4354000,324056000,56252000,427859000,27794000,33577000,17623000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2011,10889,NaN,NaN,400964,354524,1021325,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011,NaN,NaN,NaN,24326,1970,26296,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011,15598000,NaN,7168000,91351000,46786000,170298000,26581000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2011,5718,NaN,NaN,NaN,NaN,5718,9400,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
balance_df_final = balance_full_df[['year', 'CIK', 'AssetsCurrent', 'LiabilitiesCurrent']]

In [94]:
column_names = [column for column in balance_full_df.columns]

In [95]:
sorted(column_names)

['AccountsAndOtherReceivablesNetCurrent',
 'AccountsPayableAndAccruedLiabilitiesCurrent',
 'AccountsPayableCurrent',
 'AccountsPayableTradeCurrent',
 'AccountsReceivableGrossCurrent',
 'AccountsReceivableNetCurrent',
 'AccruedExpensesAndOtherCurrentLiabilities',
 'AccruedIncomeTaxesCurrent',
 'AccruedIncomeTaxesNoncurrent',
 'AccruedLiabilitiesCurrent',
 'Accruedexpensesandotherpayable',
 'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment',
 'AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax',
 'AccumulatedOtherComprehensiveIncomeLossNetOfTax',
 'AdditionalPaidInCapital',
 'AdditionalPaidInCapitalCommonStock',
 'AdvancesOnStockSubscriptions',
 'AssetRetirementObligation',
 'AssetRetirementObligationsNoncurrent',
 'Assets',
 'AssetsCurrent',
 'AssetsNoncurrent',
 'AssetsNoncurrentExcludingPropertyPlantAndEquipment',
 'AvailableForSaleSecuritiesCurrent',
 'AvailableForSaleSecuritiesNoncurrent',
 'BillingsInExcessOfCostCurrent',
 'B

In [96]:
balance_test_drop = balance_full_df.dropna(axis=1)

In [97]:
balance_full_df.isna().sum() 

year                                                                                          0
CashAndCashEquivalentsAtCarryingValue                                                         5
NotesAndLoansReceivableNetCurrent                                                            24
PrepaidExpenseAndOtherAssetsCurrent                                                          15
AssetsCurrent                                                                                 2
PropertyPlantAndEquipmentNet                                                                  4
Assets                                                                                        1
AccountsPayableCurrent                                                                        6
EmployeeRelatedLiabilitiesCurrent                                                            23
OtherLiabilitiesCurrent                                                                      19
AdvancesOnStockSubscriptions            

In [98]:
balance_test_drop

,year,LiabilitiesAndStockholdersEquity,CIK
2,2011,95169,0001162816
2,2011,5545052,0001497649
3,2011,30596000,0000718924
2,2011,106043,0001362516
15,2011,204023,0001039466
3,2011,427859000,0000068709
5,2011,1021325,0001112372
4,2011,26296,0001482179
3,2011,170298000,0000808326
5,2011,5718,0001383534


In [99]:
tsla = yf.Ticker('TSLA')
spy = yf.Ticker('SPY')

In [100]:
tsla.history()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-07-12,710.539978,719.320007,685.109985,699.210022,29310300,0,0
2022-07-13,676.500000,726.179993,675.099976,711.119995,32651500,0,0
2022-07-14,704.690002,715.960022,688.000000,714.940002,26185800,0,0
2022-07-15,720.000000,730.869995,710.669983,720.200012,23227700,0,0
2022-07-18,734.809998,751.549988,718.809998,721.640015,27512500,0,0
2022-07-19,735.000000,741.419983,710.929993,736.590027,26963400,0,0
2022-07-20,740.349976,751.989990,730.450012,742.500000,29621400,0,0
2022-07-21,765.320007,819.799988,764.599976,815.119995,47344100,0,0
2022-07-22,828.659973,842.359985,812.140015,816.729980,34490900,0,0


In [101]:
spy.history()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-07-12,383.649994,386.160004,378.989990,380.829987,62219200,0,0
2022-07-13,375.100006,381.920013,374.660004,378.829987,84224600,0,0
2022-07-14,373.609985,379.049988,371.040009,377.910004,89704800,0,0
2022-07-15,382.549988,385.250000,380.540009,385.130005,79060400,0,0
2022-07-18,388.380005,389.089996,380.660004,381.950012,63203600,0,0
2022-07-19,386.079987,392.869995,385.390015,392.269989,78506000,0,0
2022-07-20,392.470001,396.260010,391.029999,394.769989,71843800,0,0
2022-07-21,394.160004,398.839996,391.630005,398.790009,64903900,0,0
2022-07-22,398.920013,400.179993,392.750000,395.089996,72197300,0,0


In [6]:
#### convert XBRL-JSON of income statement to pandas dataframe
def get_income_statement(xbrl_json):
    income_statement_store = {}

    # iterate over each US GAAP item in the income statement
    for usGaapItem in xbrl_json['StatementsOfIncome']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfIncome'][usGaapItem]:
            # only consider items without segment. not required for our analysis.
            if 'segment' not in fact:
                index = fact['period']['startDate'] + '-' + fact['period']['endDate']
                # ensure no index duplicates are created
                if index not in indicies:
                    values.append(fact['value'])
                    indicies.append(index)                    

        income_statement_store[usGaapItem] = pd.Series(values, index=indicies) 

    income_statement = pd.DataFrame(income_statement_store).reset_index()
    income_statement['index'] = income_statement['index'].str.slice(-10, -6)
    income_statement = income_statement.rename(columns={'index': 'year'})
    income_statement = income_statement.sort_values(by='year')
    income_statement['CIK'] = xbrl_json['CoverPage']['EntityCentralIndexKey']
    income_statement = pd.DataFrame(income_statement.iloc[-1, :]).T
    # switch columns and rows so that US GAAP items are rows and each column header represents a date range
    return income_statement



In [103]:
get_income_statement(cleaned_list_xbrl[10])

,year,SalesRevenueGoodsNet,CostOfGoodsSold,GrossProfit,OfficersCompensation,GeneralAndAdministrativeExpense,OperatingExpenses,OperatingIncomeLoss,InterestExpense,IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments,DiscontinuedOperationIncomeLossFromDiscontinuedOperationBeforeIncomeTax,IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,IncomeTaxExpenseBenefit,NetIncomeLoss,EarningsPerShareBasicAndDiluted,WeightedAverageNumberOfDilutedSharesOutstanding,CIK
4,2011,116732,72714,44018,18000,70243,88243,-44225,3109,-47334,-60495,-107829,0,-107829,-0.03,3796386,0001482179


In [104]:
cleaned_list_xbrl[0]

{'CoverPage': {'EntityRegistrantName': 'NEAH POWER SYSTEMS, INC.',
  'DocumentType': '10-K',
  'DocumentPeriodEndDate': '2011-09-30',
  'AmendmentFlag': 'false',
  'EntityCentralIndexKey': '0001162816',
  'CurrentFiscalYearEndDate': '--09-30',
  'EntityCommonStockSharesOutstanding': {'unitRef': 'Shares',
   'decimals': 'INF',
   'period': {'instant': '2011-12-23'},
   'value': '182793529'},
  'EntityPublicFloat': {'unitRef': 'USD',
   'decimals': 'INF',
   'period': {'instant': '2011-03-31'},
   'value': '800000'},
  'EntityFilerCategory': 'Smaller Reporting Company',
  'EntityCurrentReportingStatus': 'Yes',
  'EntityVoluntaryFilers': 'No',
  'EntityWellKnownSeasonedIssuer': 'No',
  'DocumentFiscalYearFocus': '2011',
  'DocumentFiscalPeriodFocus': 'FY'},
 'StatementsOfIncome': {'Revenues': [{'xsi:nil': 'true',
    'unitRef': 'USD',
    'period': {'startDate': '2010-10-01', 'endDate': '2011-09-30'}},
   {'xsi:nil': 'true',
    'unitRef': 'USD',
    'period': {'startDate': '2009-10-01', 

In [105]:
full_list2 = []

for xbrl in cleaned_list_xbrl:
    try:
        full_list2.append(get_income_statement(xbrl))
    except:
        continue
len(full_list2)    


/var/folders/lz/58xhrs5n36n2ng9kp4lk67t80000gp/T/ipykernel_2370/1613999567.py:19: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  income_statement_store[usGaapItem] = pd.Series(values, index=indicies)
/var/folders/lz/58xhrs5n36n2ng9kp4lk67t80000gp/T/ipykernel_2370/1613999567.py:19: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  income_statement_store[usGaapItem] = pd.Series(values, index=indicies)
/var/folders/lz/58xhrs5n36n2ng9kp4lk67t80000gp/T/ipykernel_2370/1613999567.py:19: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  income_statement_store[usGaapItem] = pd.Series(values, index=indicies)
/var/folders/lz/58xhrs5n36n2ng9kp4lk67t80000gp/T/ipy

18

In [106]:
income_full_df = pd.concat(full_list2)

In [107]:
income_full_df

,year,OilAndGasSalesRevenue,LeaseOperatingExpense,ProductionTaxExpense,DepreciationDepletionAndAmortization,AssetRetirementObligationAccretionExpense,GeneralAndAdministrativeExpense,OtherExpenses,CostsAndExpenses,OperatingIncomeLoss,...,OtherNonoperatingExpense,InvestmentIncomeNonoperating,OtherIncome,RentalIncomeNonoperating,DeferredIncomeTaxExpenseBenefit,OtherComprehensiveIncomeLossNetOfTaxPeriodIncreaseDecrease,AssetImpairmentCharges,ImpairmentOfIntangibleAssetsFinitelived,LitigationLossProvision,GainLossOnSaleOfPropertyPlantEquipment
0,2011,1899584,200742,87217,264657,3260,987778,25526,1569180,330404,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1425065000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2011,NaN,NaN,NaN,NaN,NaN,44230000,NaN,NaN,73194000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-465636,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011,NaN,NaN,NaN,NaN,NaN,70243,NaN,NaN,-44225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-32527000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2243835,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-178949160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61203000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2597000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
income_full_df.isna().sum()

year                                                                                                            0
OilAndGasSalesRevenue                                                                                          17
LeaseOperatingExpense                                                                                          17
ProductionTaxExpense                                                                                           17
DepreciationDepletionAndAmortization                                                                           16
AssetRetirementObligationAccretionExpense                                                                      17
GeneralAndAdministrativeExpense                                                                                11
OtherExpenses                                                                                                  16
CostsAndExpenses                                                                        

In [109]:
income_df_final = income_full_df[['OperatingIncomeLoss', 'NetIncomeLoss', 'CIK', 'year']]

In [7]:
def get_cash_flow_statement(xbrl_json):
    cash_flows_store = {}

    for usGaapItem in xbrl_json['StatementsOfCashFlows']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfCashFlows'][usGaapItem]:        
            # only consider items without segment.
            if 'segment' not in fact:
                # check if date instant or date range is present
                if "instant" in fact['period']:
                    index = fact['period']['instant']
                else:
                    index = fact['period']['startDate'] + '-' + fact['period']['endDate']

                # avoid duplicate indicies with same values
                if index in indicies:
                    continue

                if "value" not in fact:
                    values.append(0)
                else:
                    values.append(fact['value'])

                indicies.append(index)                    

        cash_flows_store[usGaapItem] = pd.Series(values, index=indicies) 


    cash_flows = pd.DataFrame(cash_flows_store).reset_index()
    cash_flows['index'] = cash_flows['index'].str.slice(-10, -6)
    cash_flows = cash_flows.rename(columns={'index': 'year'})
    cash_flows = cash_flows.sort_values(by='year')
    cash_flows['CIK'] = xbrl_json['CoverPage']['EntityCentralIndexKey']
    cash_flows = pd.DataFrame(cash_flows.iloc[-2, :]).T
    return cash_flows

In [111]:
get_cash_flow_statement(cleaned_list_xbrl[14])

,year,NetIncomeLoss,IncreaseDecreaseInAccountsPayable,NetCashProvidedByUsedInOperatingActivities,ProceedsFromIssuanceOfCommonStock,NetCashProvidedByUsedInFinancingActivities,CashAndCashEquivalentsPeriodIncreaseDecrease,CashAndCashEquivalentsAtCarryingValue,CIK
8,2011,-28004,4400,-23604,25000,25000,1396,NaN,0001383528


In [112]:
full_list3 = []

for xbrl in cleaned_list_xbrl:
    try:
        full_list3.append(get_cash_flow_statement(xbrl))
    except:
        continue
len(full_list3)  

/var/folders/lz/58xhrs5n36n2ng9kp4lk67t80000gp/T/ipykernel_2370/2032391621.py:28: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  cash_flows_store[usGaapItem] = pd.Series(values, index=indicies)


20

In [113]:
cashflow_full_df = pd.concat(full_list3)
cashflow_full_df.head()

,year,NetIncomeLoss,DepreciationDepletionAndAmortization,AmortizationOfFinancingCosts,AssetRetirementObligationAccretionExpense,AmortizationOfDebtDiscountPremium,IncreaseDecreaseInParticipationLiabilities,ShareBasedCompensation,GainLossOnDerivativeInstrumentsNetPretax,IncreaseDecreaseInAccountsReceivable,...,ImpairmentOfIntangibleAssetsIndefinitelivedExcludingGoodwill,InventoryLIFOReserve,InventoryLIFOReservePeriodCharge,LoansAndLeasesReceivableRelatedParties,AccountsPayableOther,TaxesPayable,CustomerAdvancesAndDeposits,EquipmentExpense,CashDividendsPaidToParentCompanyByConsolidatedSubsidiaries,DueFromOtherRelatedParties
3,2011,-1109397,264657,205697,3260,280256,108503,258731,-761076,-252247,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011,NaN,NaN,NaN,NaN,NaN,NaN,3909269,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2011,50942000,12894000,NaN,NaN,NaN,NaN,2701000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2011,NaN,56424,NaN,NaN,NaN,NaN,NaN,NaN,-35540,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2011,-107829,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-53832,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
cashflow_full_df.isna().sum()

year                                                                                              0
NetIncomeLoss                                                                                     7
DepreciationDepletionAndAmortization                                                             12
AmortizationOfFinancingCosts                                                                     14
AssetRetirementObligationAccretionExpense                                                        18
AmortizationOfDebtDiscountPremium                                                                16
IncreaseDecreaseInParticipationLiabilities                                                       19
ShareBasedCompensation                                                                            7
GainLossOnDerivativeInstrumentsNetPretax                                                         17
IncreaseDecreaseInAccountsReceivable                                                              8


In [115]:
cashflow_df_final = cashflow_full_df[['year', 'CIK', 'NetCashProvidedByUsedInOperatingActivities', 'NetCashProvidedByUsedInInvestingActivities', 'NetCashProvidedByUsedInFinancingActivities', 'CashAndCashEquivalentsPeriodIncreaseDecrease']]

In [198]:
financials_df_final = balance_df_final.merge(cashflow_df_final, how='inner', on=['CIK', 'year']).merge(income_df_final, how='inner', on=['CIK', 'year'])

In [199]:
financials_df_final

,year,CIK,AssetsCurrent,LiabilitiesCurrent,NetCashProvidedByUsedInOperatingActivities,NetCashProvidedByUsedInInvestingActivities,NetCashProvidedByUsedInFinancingActivities,CashAndCashEquivalentsPeriodIncreaseDecrease,OperatingIncomeLoss,NetIncomeLoss
0,2011,0001497649,714818,255087,477266,-3398308,3217349,296307,330404,-1109397
1,2011,0000068709,324056000,195614000,42958000,-10145000,-5817000,27484000,73194000,50942000
2,2011,0001112372,400964,1476517,-366870,-398116,735952,-15827,-465636,NaN
3,2011,0001482179,24326,NaN,11884,0,-53203,NaN,-44225,-107829
4,2011,0000808326,91351000,67058000,-6289000,-15286000,17887000,-4346000,-32527000,-34219000
5,2011,0000030831,15644122,6207295,3766380,-1780194,-1745126,367874,2243835,1348734
6,2011,0001135202,355328,165547,-1482868,119498,728286,-635084,-178949160,-178921042
7,2011,0001512574,240849000,136082000,72346000,-23087000,-26765000,22597000,61203000,16491000
8,2011,0000017485,16095000,6230000,-4494000,-832000,914000,NaN,2597000,259000
9,2011,0001502152,21942,31625,-52480,0.0,65500,13020,-54105,-54105


In [8]:
mapper = StockMapper()

In [9]:
map_stock = mapper.cik_to_tickers

In [10]:
financials_df_final['ticker'] = financials_df_final['CIK'].map(map_stock)

NameError: name 'financials_df_final' is not defined

In [169]:
financials_df_final = financials_df_final.dropna()

In [203]:
financials_df_final

,year,CIK,AssetsCurrent,LiabilitiesCurrent,NetCashProvidedByUsedInOperatingActivities,NetCashProvidedByUsedInInvestingActivities,NetCashProvidedByUsedInFinancingActivities,CashAndCashEquivalentsPeriodIncreaseDecrease,OperatingIncomeLoss,NetIncomeLoss,ticker
0,2011,0001497649,714818,255087,477266,-3398308,3217349,296307,330404,-1109397,{GSTX}
1,2011,0000068709,324056000,195614000,42958000,-10145000,-5817000,27484000,73194000,50942000,NaN
2,2011,0001112372,400964,1476517,-366870,-398116,735952,-15827,-465636,NaN,NaN
3,2011,0001482179,24326,NaN,11884,0,-53203,NaN,-44225,-107829,NaN
4,2011,0000808326,91351000,67058000,-6289000,-15286000,17887000,-4346000,-32527000,-34219000,{EMKR}
5,2011,0000030831,15644122,6207295,3766380,-1780194,-1745126,367874,2243835,1348734,{DYSL}
6,2011,0001135202,355328,165547,-1482868,119498,728286,-635084,-178949160,-178921042,NaN
7,2011,0001512574,240849000,136082000,72346000,-23087000,-26765000,22597000,61203000,16491000,NaN
8,2011,0000017485,16095000,6230000,-4494000,-832000,914000,NaN,2597000,259000,NaN
9,2011,0001502152,21942,31625,-52480,0.0,65500,13020,-54105,-54105,{GSTC}


In [171]:
financials_df_final['ticker'] = financials_df_final['ticker'].map(str)

In [173]:
financials_df_final['ticker'] = financials_df_final['ticker'].str.slice(2, -2)

In [174]:
financials_df_final

,year,CIK,AssetsCurrent,LiabilitiesCurrent,NetCashProvidedByUsedInOperatingActivities,NetCashProvidedByUsedInInvestingActivities,NetCashProvidedByUsedInFinancingActivities,CashAndCashEquivalentsPeriodIncreaseDecrease,OperatingIncomeLoss,NetIncomeLoss,ticker
0,2011,0001497649,714818,255087,477266,-3398308,3217349,296307,330404,-1109397,GSTX
4,2011,0000808326,91351000,67058000,-6289000,-15286000,17887000,-4346000,-32527000,-34219000,EMKR
5,2011,0000030831,15644122,6207295,3766380,-1780194,-1745126,367874,2243835,1348734,DYSL
9,2011,0001502152,21942,31625,-52480,0.0,65500,13020,-54105,-54105,GSTC


In [8]:
import datetime

start = datetime.datetime.strptime("2009-01-01", "%Y-%m-%d")
end = datetime.datetime.strptime("2012-12-31", "%Y-%m-%d")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
    
cleaned_dates = []
for date in date_generated[::7]:
    cleaned_dates.append(date.strftime("%Y-%m-%d"))
    
cleaned_dates[:5]

['2009-01-01', '2009-01-08', '2009-01-15', '2009-01-22', '2009-01-29']

In [10]:
cleaned_date_range = [f"{cleaned_dates[i]} TO {cleaned_dates[i+1]}" for i in range(len(cleaned_dates) - 1)]
shorter_range = cleaned_date_range[:1000]
cleaned_date_range[:5]

['2009-01-01 TO 2009-01-08',
 '2009-01-08 TO 2009-01-15',
 '2009-01-15 TO 2009-01-22',
 '2009-01-22 TO 2009-01-29',
 '2009-01-29 TO 2009-02-05']

In [30]:
from sec_api import QueryApi

api_key = '88140c3a554da7220ba7453a5e1ea2221c5f1e0e42c3293bbe75857245774e2e'

# get your API key at https://sec-api.io
query_api = QueryApi(api_key=api_key)

# fetch all 10-K filings for all public companys from 2009-2012
query = {
    "query": {
        "query_string": {
            "query": "filedAt:[2009-01-01 TO 2012-12-31] AND (formType:\"10-K\")"
        }
    },
    "size": "200",
    "sort": [{ "filedAt": { "order": "desc" } }]
}

query_result_1000 = query_api.get_filings(query)

In [11]:
full_query = []

for drange in shorter_range:
    query = {
        "query": {
            "query_string": {
                "query": f"filedAt:[{drange}] AND (formType:\"10-K\")"
            }
        },
        "size": "200",
        "sort": [{ "filedAt": { "order": "desc" } }]
    }

    full_query.append(query_api.get_filings(query)['filings'])

In [36]:
query_result_1000

{'total': {'value': 10000, 'relation': 'gte'},
 'query': {'from': 0, 'size': 200},
 'filings': [{'ticker': 'XSNX',
   'formType': 'NT 10-K',
   'accessionNo': '0001391609-12-000387',
   'cik': '1039466',
   'companyNameLong': 'XSUNX INC (Filer)',
   'effectivenessDate': '2013-01-02',
   'companyName': 'XSUNX INC',
   'linkToFilingDetails': 'https://www.sec.gov/Archives/edgar/data/1039466/000139160912000387/nt10k_xsunx.htm',
   'description': 'Form NT 10-K - Notification of inability to timely file Form 10-K 405, 10-K, 10-KSB 405, 10-KSB, 10-KT, or 10-KT405',
   'linkToTxt': 'https://www.sec.gov/Archives/edgar/data/1039466/000139160912000387/0001391609-12-000387.txt',
   'filedAt': '2012-12-31T18:05:51-05:00',
   'documentFormatFiles': [{'sequence': '1',
     'size': '19546',
     'documentUrl': 'https://www.sec.gov/Archives/edgar/data/1039466/000139160912000387/nt10k_xsunx.htm',
     'description': 'FORM NT 10-K XSUNX, INC.',
     'type': 'NT 10-K'},
    {'sequence': '\xa0',
     'size

In [47]:
accession_numbers = []

# extract accession numbers of each filing
for item in query_result_1000['filings']:
    accession_numbers.append(item['accessionNo']);

In [48]:
len(accession_numbers)

200

In [43]:
import time

# get XBRL-JSON for a given accession number
def get_xbrl_json(accession_no, retry = 0):
    request_url = xbrl_converter_api_endpoint + "?accession-no=" + accession_no + "&token=" + api_key

    # linear backoff in case API fails with "too many requests" error
    try:
      response_tmp = requests.get(request_url)
      xbrl_json = json.loads(response_tmp.text)
    except:
      if retry > 5:
        raise Exception('API error')
      
      # wait 500 milliseconds on error and retry
      time.sleep(0.5) 
      return get_xbrl_json(accession_no, retry + 1)

    return xbrl_json

In [ ]:
list_xbrl = []
counter = 0

for acc_no in accession_numbers:
    print(f"processing {counter}")
    list_xbrl.append(get_xbrl_json(acc_no))
    counter += 1

processing 0
processing 1
processing 2
processing 3
processing 4
processing 5
processing 6
processing 7
processing 8
processing 9
processing 10
processing 11
processing 12
processing 13
processing 14
processing 15
processing 16
processing 17
processing 18
processing 19
processing 20
processing 21
processing 22
processing 23
processing 24
processing 25
processing 26
processing 27
processing 28
processing 29
processing 30
processing 31
processing 32
processing 33
processing 34
processing 35
processing 36
processing 37
processing 38
processing 39
processing 40
processing 41
processing 42
processing 43
processing 44
processing 45
processing 46
processing 47
processing 48
processing 49
processing 50
processing 51
processing 52
processing 53
processing 54
processing 55
processing 56
processing 57
processing 58
processing 59
processing 60
processing 61
processing 62
processing 63
processing 64
processing 65
processing 66
processing 67
processing 68
processing 69
processing 70
processing 71
pr

In [17]:
cleaned_list_xbrl = []

for xbrl in list_xbrl:
    if 'message' in xbrl.keys():
        pass
    else:
        cleaned_list_xbrl.append(xbrl)

In [18]:
len(list_xbrl)

250

In [19]:
len(cleaned_list_xbrl)

0

In [20]:
list_xbrl

[{'message': 'XBRL conversion started, but the processing has not been completed. Please try again after 60 seconds.'},
 {'message': 'XBRL conversion started, but the processing has not been completed. Please try again after 60 seconds.'},
 {'message': 'XBRL conversion started, but the processing has not been completed. Please try again after 60 seconds.'},
 {'message': 'XBRL conversion started, but the processing has not been completed. Please try again after 60 seconds.'},
 {'message': 'XBRL conversion started, but the processing has not been completed. Please try again after 60 seconds.'},
 {'message': 'XBRL conversion started, but the processing has not been completed. Please try again after 60 seconds.'},
 {'message': 'XBRL conversion started, but the processing has not been completed. Please try again after 60 seconds.'},
 {'message': 'XBRL conversion started, but the processing has not been completed. Please try again after 60 seconds.'},
 {'message': 'XBRL conversion started, b

In [31]:
mapper = StockMapper()
map_stock = mapper.cik_to_tickers

In [32]:
def get_sector(ticker):
    
    """
    This function takes the ticker for a public company as a string and returns the company's sector.
    """
    
    tick = yf.Ticker(ticker)
    return tick.get_info()['sector']

In [44]:
def stock_price_start_end(ticker, year):
    
    """
    This function takes the ticker for a public company and a start year to begin tracking the company. 
    The function returns the stock price from the beginning of the specified year as well as the stock 
    price exactly 10 years later."""
    
    tick = yf.Ticker(ticker)
    start_price = tick.history(start=f'{year}-01-01')['Close'][0]
    end_year = str(int(year)+10)
    final_price = tick.history(start=f'{end_year}-01-01')['Close'][0]
    return start_price, final_price

In [45]:
def financials(list_of_xbrl):
    balance_full_list = []

    for xbrl in cleaned_list_xbrl:
        try:
            balance_full_list.append(get_balance_sheet(xbrl))
        except:
            continue   
    balance_full_df = pd.concat(balance_full_list)
    balance_df_final = balance_full_df[['year', 'CIK', 'AssetsCurrent', 'LiabilitiesCurrent']]
    
    income_full_list = []

    for xbrl in cleaned_list_xbrl:
        try:
            income_full_list.append(get_income_statement(xbrl))
        except:
            continue   
    income_full_df = pd.concat(income_full_list)
    income_df_final = income_full_df[['OperatingIncomeLoss', 'NetIncomeLoss', 'CIK', 'year']]
    
    cashflow_full_list = []

    for xbrl in cleaned_list_xbrl:
        try:
            cashflow_full_list.append(get_cash_flow_statement(xbrl))
        except:
            continue   
    cashflow_full_df = pd.concat(cashflow_full_list)
    cashflow_df_final = cashflow_full_df[['year', 'CIK', 'NetCashProvidedByUsedInOperatingActivities', 'NetCashProvidedByUsedInInvestingActivities', 'NetCashProvidedByUsedInFinancingActivities', 'CashAndCashEquivalentsPeriodIncreaseDecrease']]
    
    final_financials = balance_df_final.merge(cashflow_df_final, how='inner', on=['CIK', 'year']).merge(income_df_final, how='inner', on=['CIK', 'year'])
    final_financials['ticker'] = final_financials['CIK'].map(map_stock)
    final_financials.dropna(inplace=True)
    
    final_financials['ticker'] = final_financials['ticker'].map(str).str.slice(2, -2)
    
    final_financials['sector'] = final_financials['ticker'].map(get_sector)
    
    final_financials['stock_start'], final_financials['stock_end'] = final_financials.apply(lambda final_financials: 
                                                                                            stock_price_start_end(final_financials['ticker'], 
                                                                                                                  final_financials['year']), axis=1)
    
    return final_financials
    

In [46]:
financials(cleaned_list_xbrl)

ValueError: not enough values to unpack (expected 2, got 1)

In [40]:
appl = yf.Ticker('AAPL')

In [41]:
appl.get_info()['sector']

'Technology'

In [52]:
appl.history(start='2011-01-01')['Close'][0]

10.049904823303223

In [53]:
appl.history(start='2021-01-01')['Close'][0]

128.0870819091797